# Introducción a Bases de datos y SQL





---



### Recapitulación de la clase SQL

En la clase anterior tuvimos una introducción a los conceptos claves de SQL:
1. ¿Qué es una base de datos? ¿Para qué nos sirve?
2. ¿Qué NO es una base de datos? {Excel por ejemplo}
3. ¿Qué es un DBMS? ¿Qué es un motor de bases de datos?
4. Tipos de bases de datos
  - Pros y contras de las mismas
5. Tipos de datos
6. Diseño y creación de una base de datos
7. Introducción al lenguaje SQL
  - Como se conforma un comando SELECT

## Entremos en profundidad en dos puntos clave: 

#### Tipos de datos: ¿por qué son tan importantes?

A la hora de crear el esquema de nuestras tablas, es muy importante que conozcamos en detalle nuestros datos:
- ¿Qué volumen manejaremos?
- ¿Cuales y como son las fuentes de datos que recibiremos?
- ¿Hay cambios? ¿Con qué frecuencia?

### Revisando la creación de la tabla 'Customers' de la clase pasada:



```
CREATE TABLE IF NOT EXISTS bootcamp.Customers (
  customer_id INT NOT NULL,
  customer_name VARCHAR(50) NOT NULL,
  fecha_inicio DATE NOT NULL,
  fecha_fin DATE,
  PRIMARY KEY (customer_id)
  );
```

### ¿Cual es la diferencia entre estas dos formas de crear la tabla?

```
CREATE TABLE IF NOT EXISTS bootcamp.Customers (
  customer_id BIGINT NOT NULL,
  customer_name TEXT NOT NULL,
  fecha_inicio DATE NOT NULL,
  fecha_fin DATE ,
  PRIMARY KEY (customer_id)
  );
```

A priori, las diferencias no parecen muy grandes:
- En el primer caso tenemos que:
  - customer_id será de tipo INT
  - customer_name será de tipo VARCHAR(50)
  - fecha_inicio y fecha_fin serán de tipo DATE

- En el segundo caso tenemos que:
  - customer_id sería de tipo BIGINT
  - customer_name sería de tipo TEXT





Éste paso, que en ocasiones puede aparentar trivial, es muy importante a la hora de administrar nuestras bases de datos por distintas razones.

1. Elegir INT por sobre BIGINT haría que cada valor de 'customer_id' sea más ligero de almacenar pero nos "ata" al rango de valores que el tipo de dato INT implica.
2. En el caso de 'customer_name' sucede algo similar, si nosotros optamos por elegir el tipo de dato VARCHAR(50) cada valor de dicha columna ocupará 2 bytes + el número de caracteres pero tiene la contra de que si un cliente tuviera un nombre muy largo, superior a los 50 caracteres no podríamos ingresarlo directamente a la tabla.





---



---



### Repaso de la anatomía de un comando SELECT
De la clase anterior sabemos que

La sentencia SELECT se compone de varias cláusulas:

* SELECT

* FROM

* WHERE

* GROUP BY

* ORDER BY

* HAVING


Sin embargo, algo que no mencionamos es que el ORDEN en el que escribamos nuestras clausulas es muy importante. Si no respetamos dicho orden el comando puede fallar por completo.

En éste sentido, les presentamos una regla nemotécnica para recordar el orden en el que deben ir las clausulas.

***'So few workers go home on time'***

So = SELECT

Few = FROM

Workers = WHERE

Go = GROUPBY

Home = HAVING

On = ORDERBY

time

Ejemplos: 


Ésta sentencia SQL es incorrecta y arrojaría un error
```
SELECT columna1, COUNT(columna2)
FROM tabla
GROUP BY columna1
WHERE columna2 = 1
```

Por su parte la siguiente sentencia funcionaría:

```
SELECT columna1, COUNT(columna2)
FROM tabla
WHERE columna2 = 1
GROUP BY columna1
```

Como se ve, la diferencia aparenta ser pequeña pero puede darnos muchos dolores de cabeza si no prestamos atención al orden de las clausulas.




---



---



## Introducción a las vistas o views

Una vista es una tabla "virtual" basada en el resultado de una consulta de tipo "SELECT" a una tabla.

Existen varias razones por las cuales las vistas nos pueden resultar útiles:
- Por temas de seguridad, podríamos necesitar que un grupo de usuarios tenga acceso exclusivamente a algunos datos y no otros dentro de una tabla.
- Por temas de conveniencia, nos podría interesar filtrar las columnas que queremos que un usuario vea.
- Etc,.


Crear una vista es muy similar a crear una tabla con la importante diferencia que tenemos que especificar un comando SELECT a la hora de crearla.


Como ejemplo, vamos a crear una vista en la cual queremos mostrar datos de la tabla 'bootcamp.Customers' que trabajamos la clase anterior, con la condición de que solo vamos a mostrar los clientes cuyo nombre contenga la letra 'u'.

En este caso, vamos a crear una vista con las mismas columnas que la tabla original pero filtrando los registros o las filas.
```
CREATE VIEW vista_ejemplo AS
SELECT *
FROM bootcamp.Customers c
WHERE c.customer_name LIKE '%u%'
```

De la misma manera podríamos crear una vista que contenga las mismas filas que la tabla original pero filtrando las columnas que queremos mostrar.

```
CREATE VIEW vista_ejemplo_2 AS
SELECT c.customer_name, c.fecha_inicio
FROM bootcamp.Customers c
```

Por último, lógicamente ésto se puede complicar tanto como lo deseemos. Pongamos como ejemplo que algún usuario nos solicita tener una vista disponible que muestre lo siguiente:

- Solamente el nombre de los usuarios
- Que se dieron de alta el día de ayer

```
CREATE VIEW vista_ejemplo_3 AS
SELECT c.customer_name, c.fecha_inicio
FROM bootcamp.Customers c
WHERE c.fecha_inicio = current_date - 1
```




---



---



---



## Introducción a las subconsultas o subqueries

Como indica su nombre, una subconsulta o subquery, es una query de SQL anidada dentro de una query mayor.

Hasta el momento trabajamos con consultas o queries relativamente sencillas, pero a medida que se complica el trabajo nos vamos a encontrar con situaciones en las que las subqueries resultan muy útiles.

Una subquery puede estar anidada dentro de una consulta de tipo SELECT, INSERT, UPDATE o DELETE pero se utiliza -generalmente- en conjunto con la clausula WHERE dentro de una sentencia SELECT. 

Veamoslo con un ejemplo para entenderlo mejor:
Retomando la siguiente tabla 'bootcamp.Orders', tenemos 


```
CREATE TABLE IF NOT EXISTS bootcamp.Orders (
  order_id INT NOT NULL,
  customer_id INT NOT NULL,
  order_date DATE NOT NULL,
  order_price DECIMAL(8,2),
  PRIMARY KEY (order_id),
FOREIGN KEY (customer_id) REFERENCES bootcamp.Customers(customer_id)
  );
```

Ahora supongamos que tenemos que consultar esa tabla y retornar los registros donde el 'order_price' sea máximo. ¿Cómo podemos lograr ésto si de antemano no sabemos cual es el precio máximo?


```
SELECT * 
FROM bootcamp.Orders
WHERE order_price = (SELECT MAX(order_price) FROM Orders)
```

Como mencionamos antes, ésto también puede utilizarse en consultas del tipo INSERT, por ejemplo.

En un nuevo ejemplo, podemos tener el siguiente caso:
- Tenemos una tabla llamada 'bootcamp.Orders_antiguas' con el mismo esquema que 'bootcamp.Orders'
- Queremos insertar los registros de Orders_antiguas a la nueva tabla "bootcamp.Orders" pero solo a partir de cierta fecha.
Utilizar una subconsulta aquí nos facilitaría el trabajo


```
INSERT INTO bootcamp.Orders
SELECT * 
FROM bootcamp.Orders_antiguas o
WHERE o.order_date >= '2021-01-01'
```



Y así como vimos con la introducción a las vistas, ésto puede extenderse y complicarse tanto como lo necesitemos. De todas formas, acá cabe una aclaración no menor:
- Las subqueries o subconsultas NO son la forma más eficiente en todas las circunstancias. Por lo que si vemos que estamos "abusando" o reutilizando muchas veces las mismas subconsultas, probablemente sea conveniente re-pensar nuestras queries o diagrama de datos y analizar de que manera podemos mejorarlo.





---



---



---



## Introducción a los Joins

Los "joins" son una de las partes más importantes del lenguaje SQL.
En términos sencillos nos permiten unir y cruzar datos entre tablas (o incluso cruzar una tabla consigo misma)

En la última parte de la clase anterior vimos un caso donde queríamos seleccionar un "Total Gastado" que se obtenía de la tabla 'bootcamp.Orders' y traer además el nombre de quien había gastado ese monto - dato que existía en la tabla 'bootcamp.Customers'.

Para este tipo de situaciones y muchas otras que se encuentran en el día a día trabajando con bases de datos, los joins son la herramienta principal.

En éste sentido, existen 4 tipos de joins:

- (INNER) JOIN: Devuelve los registros que existan en ambas tablas exclusivamente.
  - En el ejemplo de la clase pasada donde queríamos "unir" datos de las tablas 'Orders' y 'Customers', éste join nos retornaría exclusivamente registros en donde un 'Customer' tiene una 'Order' y viceversa.

```
SELECT *
FROM bootcamp.Orders o
INNER JOIN bootcamp.Customers c
ON o.customer_id = c.customer_id 
-- Solo retornará registros donde exista la coincidencia o.customer_id = c.customer_id 
```


- LEFT (OUTER) JOIN: Devuelve todos los registros de la tabla que esté 'a la izquierda' del join y además aquellos que coincidan con la tabla 'a la derecha'.
  - Continuando con el ejemplo anterior

```
SELECT *
FROM bootcamp.Orders o
LEFT JOIN bootcamp.Customers c
ON o.customer_id = c.customer_id 
-- Retornará TODOS los registros de bootcamp.Orders 
-- y además aquellos de bootcamp.Customers donde exista la coincidencia o.customer_id = c.customer_id 
```
- RIGHT (OUTER) JOIN: Es la versión "complementaria" al LEFT JOIN
<br><br/>
- FULL (OUTER) JOIN: Devuelve todos los registros de ambas tablas, uniendo aquellos en los que haya coincidencia.




---



---



## Introducción a la normalización

Entender y cementar todos los conocimientos y prácticas relacionados a la normalización de una base de datos escapa al alcance del curso.

De todas formas ésta pretende ser una breve introducción al tema para entender las buenas prácticas y las convenciones que hacen a la correcta administración de las bases de datos y las tablas que viven en ella.

En términos concretos la normalización se entiende como una técnica de organización de datos en una base que consiste en descomponer las tablas para eliminar la redundancia. Es un proceso de multiples pasos para llevar la data a una forma tabular, eliminando duplicación en las tablas relacionales.

Si no prestamos atención a la normalización podemos caer rápidamente en una situación en la que la base se vuelve lenta y vulnerable a la inconsistencia de los datos.

La normalización es un proceso en constante desarrollo a la que se suman nuevas prácticas constantemente. Por tanto, nos vamos a enfocar en las llamadas "primeras dos formas de normalización".

### 1NF - First Normal Form o Primera forma normal:
Para 'cumplir' con la primera forma normal se deben:
- Remover grupos repetidos de las tablas y registros con multiples valores.
- Crear tablas separadas por cada grupo de data relacionada
- Identificar los distintos grupos de datos relacionados mediante primary keys.


### 2NF - Second Normal Form o Segunda forma normal:
Para acatar con la segunda forma normal debemos:
- Cumplir con la primera forma
- Asegurarnos que no exista dependencia parcial
